# Taking a look at stocks
Stocks are the mobility choices and are represented by teh variable `stock_X`, where `X` is the type of mobility, and it can be:
- `C`: combustion car
- `E`: electric car
- `N`: non-motorized.
- `P`: public transport.
- `S`: shared mobility.

The first goal, is to create a dataframe that captures the result of each scenario for each of these stocks. Later, I want to make a correlation matrix (if possible).

In [43]:
import numpy as np
import itertools
import pandas as pd
import warnings
import random

import datetime as dt
import time

import os
from os import listdir
from os.path import isfile,join
import matplotlib.pyplot as plt

### Path Variables

In [1]:
PATH = '/home/moni/Documents/motmo/timeSeries_files/' # original data
PATH2 = '/home/moni/Documents/motmo/data_without_hhID/' # folder in which we will store transformed data

### List of strings scenarios names

In [3]:
def list_file_names():
    file_names = [f for f in listdir(PATH2) if isfile(join(PATH2, f))]
    # file_names.remove('.~lock.timeSeries_CH0SP0SE0WE0BP0RE0CO0DI0WO0CS0.csv#')
    return file_names
def list_str_scenarios(file_names):# gets the stringignoring the csv extension
    sc_str = [name.replace(".csv", "") for name in file_names]
    sc_str = [name.replace("timeSeries_", "") for name in sc_str]
    return sc_str

### Read .csv file
Since the output form the model is 2018, we read from step 13.

In [4]:
def read_csv_clean(file_name):
    df = pd.read_csv(PATH2 + file_name)
    mask = df['step']>=13 # we start in 2018
    df = df[mask]
    # dates_list = pd.date_range(start = "2018-01-01",periods = 181,freq="2M").strftime("%b-%Y").tolist()
    # ind_name = dict(zip(list(range(0,163)),dates_list))
    df.drop(columns=df.columns[0], axis=1, inplace=True)
    return df

### Add Stocks columns

In [ ]:
def add_stocks_x(df):
    stock_index=['C','E','N','P','S'] # variable names ending
    for stock in stock_index:
        df[f'total_stck_{stock}'] = df.iloc[:,7:12].sum(axis=1)
    df['total_stck_'] = df.iloc[:,2:7].sum(axis=1)

In [26]:
test_df = read_csv_clean('timeSeries_CH0SP0SE0WE0BP0RE0CO0DI0WO0CS0.csv')
test_df = test_df.groupby(by=["step"]).sum()
# test_df['s_C_per_age'] = test_df['emissions_C'] / test_df['stock_C']
# test_df = test_df.drop(columns = ['reID','emissions_S','emissions_C','emissions_E','emissions_N','emissions_P','total_emissions','s_C_per_capita'])
s_indx = ['C','E','N','P','S']

for s in s_indx:
    test_df[f'st_{s}_perAgent'] = test_df[f'emissions_{s}'] / test_df[f'stock_{s}']
test_df

,reID,emissions_C,emissions_E,emissions_N,emissions_P,emissions_S,stock_C,stock_E,stock_N,stock_P,stock_S,total_emissions,st_C_perAgent,st_E_perAgent,st_N_perAgent,st_P_perAgent,st_S_perAgent
step,,,,,,,,,,,,,,,,,
13,37243,1.256967e+10,7.996720e+06,8158.916221,9.928296e+08,1.080651e+07,1032614,459,104591,203387,762,1.358131e+10,12172.665854,17422.048919,0.078008,4881.480245,14181.767479
14,37243,1.256320e+10,7.996720e+06,8198.495717,9.891961e+08,1.079077e+07,1032468,459,105053,203073,760,1.357120e+10,12168.128465,17422.048919,0.078042,4871.135506,14198.376101
15,37243,1.255609e+10,7.998874e+06,8241.619981,9.855385e+08,1.075768e+07,1032244,460,105550,202803,756,1.356039e+10,12163.875288,17388.857559,0.078083,4859.585419,14229.740374
16,37243,1.254572e+10,7.998874e+06,8279.026019,9.836253e+08,1.072202e+07,1031865,460,106005,202731,752,1.354808e+10,12158.296683,17388.857559,0.078100,4851.874130,14258.001717
17,37243,1.253705e+10,7.998874e+06,8318.068869,9.809259e+08,1.059767e+07,1031539,460,106482,202591,741,1.353658e+10,12153.732032,17388.857559,0.078117,4841.902408,14301.852481
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,37243,1.284541e+10,2.714919e+08,9172.283230,3.442370e+08,3.288039e+08,1042985,49297,118901,97180,33450,1.378995e+10,12316.003919,5507.270344,0.077142,3542.261983,9829.712310
177,37243,1.283944e+10,2.787568e+08,9146.546269,3.422811e+08,3.344339e+08,1042039,50616,118576,96632,33950,1.379492e+10,12321.458783,5507.286291,0.077137,3542.108923,9850.778758
178,37243,1.283422e+10,2.860555e+08,9119.644454,3.400207e+08,3.396539e+08,1041100,51959,118233,96074,34447,1.379996e+10,12327.558970,5505.408200,0.077133,3539.154418,9860.188195


In [23]:
test_df

,step,reID,emissions_C,emissions_E,emissions_N,emissions_P,emissions_S,stock_C,stock_E,stock_N,stock_P,stock_S,total_emissions
0,13,942,4.737994e+08,1.083093e+05,253.076914,2.602194e+07,4.704557e+05,38182,7,3238,5221,27,5.004004e+08
1,13,1515,2.671367e+09,1.488357e+06,1800.147216,2.291534e+08,1.723627e+06,223145,94,22992,46917,129,2.903734e+09
2,13,1516,1.663485e+09,2.693181e+05,1167.394019,1.021371e+08,7.950865e+05,137489,17,15201,21005,66,1.766687e+09
3,13,1517,9.397048e+08,4.408532e+05,585.615003,6.569355e+07,9.988062e+05,78448,20,7438,13434,65,1.006839e+09
4,13,1518,8.329276e+07,9.490135e+03,39.905256,5.580418e+06,9.846679e+04,7423,1,523,1296,6,8.898117e+07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2683,180,2335,4.028369e+08,8.352730e+06,97.222601,1.182812e+07,3.623473e+05,35278,1472,1120,3405,31,4.233802e+08
2684,180,2336,6.856622e+08,1.561110e+07,199.451940,1.864413e+07,7.792839e+05,57966,2894,2439,5587,47,7.206969e+08
2685,180,3312,3.005697e+08,3.693528e+06,80.840364,7.866443e+06,2.586631e+05,24809,621,894,2293,18,3.123884e+08
2686,180,3562,5.171701e+08,1.464447e+07,209.852261,1.675187e+07,1.823093e+07,38123,2477,2366,4185,1622,5.667976e+08


## Sesitivity Analysis

First I want to generate the state of all possible inputs $\Omega$. I will do it in to batches of vectors of zeroes and ones and then one batch of size four, taking into account thet we cannot have more than 2 ones.

In [152]:
s3 = list(itertools.product(range(2), repeat=3))
s4 = list(itertools.product(range(2), repeat=4))
l1 = [i for i in s3 if sum(i)<3]
l2 = l1
l3 = [i for i in s4 if sum(i)<3]
L=[]
L.append(l1)
L.append(l2)
L.append(l3)
inputs = list(itertools.product(*L)) # here we have the 539 scenarios

### Variance decomposition
We see this model as a function $f(X)=Y$, where the inputs are the scenarios, so $X\in\mathbb{R}^d$ and the output is a real value, that in our case, can be the emissions or the mobility choices. First, we are going to do an analysis on (overall) emissions and when we implement it, we can use the othe outputs.

The idea consists in estimating certain variances that we are going to define later in order to compute some sesitivity indices (there are first and second order sensitivity indices). We write
$$Var(Y)=\sum_{i=1}^dV_i+\sum_{i<j}^dV_{ij}+\ldots+V_{1,2, \ldots,d}$$where
$$V_i=Var_{x_i}(E_{X_{\sim i}}(Y|X))$$where the $X_{\sim i}$ notation means the set of all variables except $x_i$.

#### First Order Index
$$S_i= \frac{V_i}{Var(Y)}$$We can interpret it as follows: "the fractional reduction in the variance of $Y$ which would be obtained on average if $X$ could be fixed".

### Estimating
The Monte Carlo method offers an estimation based on sampling. The first thing is to take $N$ pairs of samples (inputs) at random. However, in our case, there is a problem with this approximation, because later on we will create another $d$ matrices that will consist on replacing one column of $A$ with a column of $B$ given some indices. The thing is that it is very likely that this sampling will generate scenarios that are not possible due to the fact that we cannot choose in the same category 3 options turned to 1. Therefore, my idea is to first sample uniformly at random $A$, and then sample $B$ based on some probabilities that I will create accordingly.
1. First we should sample $N$ inputs (scenarios) and then create two $N\times d$ matrices $A$ and $B$ (be careful with the sampling $B$).
2. Build $d$ further $N\times d$ matrices $A_B^i$, for $i = 1,2,\ldots,d$, such that the $i$th column of $A_B^i$ is equal to the $i$th column of $B$, and the remaining columns are from $A$.
3. The $A, \,B$, and the $d$ $A_B^i$ matrices in total specify $N\cdot(d+2)$ points in the input space (one for each row).  Then we compute the corresponding $f(A)$, $f(B)$ and $f(A_B^i)$ values (that is, compute the emissions of each scenario).

To compute the variance, we can use the following estimator: $$Var_{x_i}(E_{X_{\sim i}}(Y|X))\approx \frac{1}{N}\sum_{i=1}^Nf(B)_j\left(f(A_B^i)_j-f(A)_j\right)$$

### 1. Creating A,B matrices

In [398]:
# This function returns the index in which the tuples of A
# have two ones. For example, if A = [(1,0,1),(0,0,1),(1,1,0,0)]
# then condition(A) = [0,2]
def condition(sample_A): # this is only 1 sample
    condi = [sum(i)>1 for i in sample_A]
    c_ind = [index for index, element in enumerate(condi) if element]
    return c_ind

In [417]:
def sampling(inputs, N):
    # inpts = np.array(inputs, dtype="object")
    # rng = np.random.default_rng()
    # A = rng.choice(inpts,44, replace = False) # 44 is the max number of samples we can take, Ill explain later
    # A = random.sample(inputs,44)
    # indx = [inputs.index(i) for i in A]
    # next_input = list(set(inputs)^set(A))
    j=0
    A,B = [],[]
    n_sample = inputs
    B_space = n_sample
    while B_space and j<N:
        At = random.sample(n_sample,1)
        condi = condition(At[0]) # returns if has tuples with two ones and its indices
        n_sample = [i for i in inputs if i!=At[0]] # update sample space (remove last sample of A)
        if condi:
            B_space = n_sample
            for c in condi1:
                ind0 = At[0][c].index(0) 
                B_space = [x for x in B_space if x[c][ind0]==0] # sample space with tuples where the index is 0
            Bt = random.sample(B_space,1) # take 1 sample from that space
        else:
            Bt = random.sample(n_sample,1) # Take any sample form remaining points
            # print("prueba")
        n_sample = [i for i in n_sample if i!=Bt[0]] # update sample space (remove last sample of B)
        A.append(list(itertools.chain(*At[0])))
        B.append(list(itertools.chain(*Bt[0])))
        j=j+1
    return A,B
    

In [606]:
# borrar = random.sample(inputs,1)
A1,B1 = sampling(inputs,50)
# print(A1)
A1

[[0, 0, 0, 0, 0, 0, 1, 0, 0, 1],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 1, 0, 1, 1, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 1, 0],
 [1, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 1, 0, 1, 1, 0, 1, 0],
 [0, 1, 1, 0, 0, 1, 1, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 1, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 0, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1, 1, 0],
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 1, 0, 0],
 [1, 0, 1, 1, 1, 0, 1, 0, 1, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [1, 0, 0, 1, 1, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 0],
 [1, 1, 0, 0, 1, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 1, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 0, 0, 1, 1],
 [1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 1, 1, 0, 0, 0, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
 [1, 0, 1, 1, 0, 1, 1, 0, 0, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [1, 0, 0,

In [630]:
B1

[[0, 0, 1, 0, 0, 1, 1, 0, 1, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [1, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 1, 0, 1, 1, 0, 1, 1, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 1, 0],
 [0, 1, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 0, 0, 0, 1, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 0, 0, 0, 1, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 1, 0, 0],
 [0, 0, 1, 0, 0, 1, 1, 0, 0, 1],
 [1, 0, 1, 1, 1, 0, 0, 1, 0, 0],
 [1, 0, 1, 0, 0, 1, 1, 0, 0, 0],
 [1, 0, 1, 1, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0],
 [1, 0, 0, 1, 0, 0, 0, 1, 0, 1],
 [0, 0, 0, 1, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 0, 0, 1, 0, 0, 1, 1],
 [0, 0, 1, 0, 1, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 1],
 [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
 [1, 1, 0, 0, 1, 1, 0, 1, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
 [1, 0, 1, 1, 0, 0, 0, 1, 1, 0],
 [0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
 [0, 0, 0,

## 2. The $d$ further matrices $A_B^i$

In [450]:
def mat_AB_i(A,B): 
    d = 10
    N = len(A)
    An,Bn = np.array(A), np.array(B)
    AB_mat = np.zeros((d,N,d))
    for i in range(0,d):
        AB_mat[i] = An
        AB_mat[i][:,i] = Bn[:,i]
    return AB_mat

In [612]:
# An[:,2]=Bn[:,2]
# borrar[0]
ABi = mat_AB_i(A1,B1)

In [449]:
borrar[0][:,2]

array([0., 0., 0., 0., 1., 0., 0., 1., 0., 0.])

## 3. Estimating the sensibility index
We will use the following estimate:
$$Var_{x_i}(E_{X_{\sim i}}(Y|X))\approx \frac{1}{N}\sum_{i=1}^Nf(B)_j\left(f(A_B^i)_j-f(A)_j\right)$$
But before, we need to compute the outputs. For that, we need to create a function that takes as input a boolean array (corresponding to one scenario) and converts it to the `file_name` and gets the total emissions for such input.

In [495]:
def scenario_string(boolean_scenario):
    list_events = ['CH','SP','SE','WE','BP','RE','CO','DI','WO','CS']
    # prefix = "timeSeries_"
    # sufix = ".csv"
    f_n = [l+str(s) for l,s in zip(list_events,boolean_scenario)]
    f_n = ''.join(f_n)
    file_name = f_n
    return file_name

In [496]:
te_df = pd.read_csv("total_emissions.csv")
te_df
cond = (te_df['scenario'] == "CH0SP0SE0WE0BP1RE1CO1DI1WO0CS0")
result = te_df[cond].total_emissions.values[0]
result

,scenario,total_emissions
0,CH1SP0SE1WE0BP1RE0CO0DI0WO0CS0,2.243213e+12
1,CH1SP0SE1WE1BP1RE0CO1DI1WO0CS0,2.082187e+12
2,CH0SP1SE0WE0BP0RE1CO0DI0WO1CS0,2.224649e+12
3,CH1SP1SE0WE0BP1RE0CO0DI1WO1CS0,2.206465e+12
4,CH1SP0SE1WE0BP0RE1CO0DI1WO0CS1,2.198543e+12
...,...,...
534,CH0SP0SE0WE0BP1RE1CO0DI0WO1CS0,2.217200e+12
535,CH0SP0SE0WE0BP1RE1CO1DI1WO0CS0,2.164500e+12
536,CH0SP0SE1WE0BP1RE1CO1DI0WO1CS0,2.203569e+12
537,CH0SP0SE0WE1BP0RE0CO0DI1WO0CS0,2.094284e+12


In [626]:
def get_output(M):#input is a matrix M
    out_vector = []
    N = len(M)
    for i in range(0,N): 
        string = scenario_string(M[i])
        cond = (te_df['scenario'] == string)
        print(cond)
        outp = te_df[cond].total_emissions.values[0]
        out_vector.append(outp)
    return out_vector
# get_output(A1)

In [574]:
ABi[0].astype(int).tolist()

[[0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 1, 1, 0, 1, 1, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 1, 1, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
 [0, 1, 1, 1, 0, 1, 0, 1, 0, 0]]

In [615]:
def get_var_xi(out_A,out_B,ABi,i):
    ABi = ABi[i].astype(int).tolist()
    out_ABi = get_output(ABi)
    N = len(out_A)
    # print(N)
    var = 0
    for j in range(0,N):
        var = out_B[j]*(out_ABi[j]-out_A[j])+var
    var = var/N
    return var

In [629]:
# varY = te_df.var()['total_emissions']
# varY
borrar = ABi[4].astype(int).tolist()
out_borrar = get_output(borrar)
out_borrar

0      False
1      False
2      False
3      False
4      False
       ...  
534    False
535    False
536    False
537    False
538    False
Name: scenario, Length: 539, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
534    False
535    False
536    False
537    False
538    False
Name: scenario, Length: 539, dtype: bool
0       True
1      False
2      False
3      False
4      False
       ...  
534    False
535    False
536    False
537    False
538    False
Name: scenario, Length: 539, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
534    False
535    False
536    False
537    False
538    False
Name: scenario, Length: 539, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
534    False
535    False
536    False
537    False
538    False
Name: scenario, Length: 539, dtype: bool
0      False
1      False
2      False
3      False
4      False
       ...  
53

[2248437657064.745,
 2266508254487.8354,
 2243212829935.37,
 2194573825211.7007,
 2177684607121.8604,
 2222360526095.622,
 2099561565462.0808,
 2091007616213.005,
 2156728384388.0613,
 2079357392320.873,
 2096671356719.682,
 2210405469558.123,
 2220334141511.392,
 2199405187917.782,
 2185941021236.512,
 2064614083583.3064,
 2111337123826.462,
 2214148720192.4736,
 2265776298044.3613,
 2101212709165.46,
 2098801502470.1853,
 2254005181426.941,
 2205575523122.9907,
 2224412111514.67,
 2173101453455.5083,
 2094946758854.514,
 2213851146389.928,
 2100620570864.0476,
 2117061585362.5488,
 2089712573918.8188,
 2096197992067.8752,
 2233062107300.5586,
 2227981540075.0376,
 2071002108762.215,
 2096381456176.619,
 2217315010384.762,
 2081309238012.4644,
 2247395184166.7246,
 2084668013010.2131,
 2094946758854.514,
 2190713682897.0725,
 2110943745582.4875,
 2212360670526.532,
 2113563673510.3003,
 2256493714299.764,
 2191569467613.5425,
 2105778956691.795,
 2227285908152.808,
 2096903048211.424,

In [620]:
for k in range(0,10):
    #o_A, o_B = get_output(A1), get_output(B1)
    var1 = get_var_xi(out_A=o_A.copy(),out_B=o_B.copy(),ABi=ABi.copy(),i=k)
    # get_output(ABi[0].astype(int).tolist())
    print(f"x_'{k}: "+str(var1/varY)+"/n")

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [597]:
B1

[[1, 0, 1, 0, 0, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 0, 0, 1, 0, 0, 1],
 [0, 1, 1, 0, 0, 1, 0, 0, 0, 0],
 [0, 0, 0, 0, 1, 0, 0, 0, 1, 1],
 [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 0, 0, 0, 0, 1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 0, 1, 0],
 [0, 0, 0, 0, 0, 1, 1, 1, 0, 0]]

In [599]:
ABi[0].astype(int).tolist()

[[1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
 [1, 0, 0, 0, 1, 0, 0, 1, 0, 0],
 [0, 0, 1, 1, 0, 1, 0, 0, 0, 0],
 [0, 1, 1, 0, 1, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 1, 0, 0, 0],
 [0, 1, 1, 0, 0, 0, 0, 1, 0, 1],
 [0, 1, 0, 0, 1, 1, 0, 1, 0, 0]]

In [591]:
emis_np = te_df['total_emissions'].values
varY = np.var(emis_np)
varY

4.3509533424333614e+21

In [603]:
# HAMMING DISTANCE
n = len(A1)

# def hamming_distance(a, b):
# # Determine if strings are equal in length
#     if len(a) != len(b):
#         print('String are not equal in length')
#     else:
#         return sum(abs(e1 - e2) for e1, e2 in zip(a, b))/ len(a)
#     # Input data
borrar = ABi[0].astype(int).tolist()
for k in range(0,n):
    bit_1 = A1[k]
    bit_2 = B1[k]
    # Calculate distance
    dist = hamming_distance(bit_1, bit_2)
    # Print result
    print(dist)

0.1
0.6
0.2
0.4
0.5
0.4
0.6
0.4


In [605]:
hamming_distance([1, 0, 1, 0, 1, 0, 0, 1, 0, 1],[1, 0, 1, 0, 1, 1, 0, 1, 0, 0])

0.2